In [1]:
import great_expectations as gx
import pandas as pd
import warnings
warnings.filterwarnings("ignore", message="`result_format` configured at the Validator-level*")

# Load the data
df = pd.read_csv("./data/transactions.csv")
df.index += 2
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Konvertera timestamp till string för validering
df["timestamp_str"] = df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")

# Create the ephemeral GX context
context = gx.get_context()

# Add a pandas datasource
data_source = context.data_sources.add_pandas(name="pandas")

# Add a dataframe asset
data_asset = data_source.add_dataframe_asset(name="transactions_data")

# Define the batch (entire DataFrame)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="batch_def")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

# Create the expectation suite with a name
suite = gx.core.expectation_suite.ExpectationSuite(name="transactions_suite")

# Get the validator using the suite
validator = context.get_validator(batch=batch, expectation_suite=suite)

# Add expectations
validator.expect_column_values_to_be_between("amount", min_value=0.01, max_value=100000)

# Add expectations for transaction data
validator.expect_column_values_to_not_be_null("transaction_id")
validator.expect_column_values_to_be_unique("transaction_id")
validator.expect_column_values_to_match_regex(
    "transaction_id",
    r"^[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}$"
)
validator.expect_column_value_lengths_to_equal("transaction_id", 36)

validator.expect_column_values_to_not_be_null("timestamp")
# Validera timestamp-format som sträng
validator.expect_column_values_to_match_strftime_format(
    "timestamp_str", "%Y-%m-%d %H:%M:%S"
)

validator.expect_column_values_to_be_between("amount", min_value=0.01)
validator.expect_column_values_to_not_be_null("currency")
validator.expect_column_values_to_match_regex("currency", r"^[A-Z]{3}$")

validator.expect_column_values_to_not_be_null("sender_account")
validator.expect_column_values_to_not_be_null("receiver_account")

validator.expect_column_values_to_not_be_null("sender_country")
validator.expect_column_values_to_not_be_null("receiver_country")
validator.expect_column_values_to_not_be_null("sender_municipality")
validator.expect_column_values_to_not_be_null("receiver_municipality")

validator.expect_column_values_to_not_be_null("transaction_type")

validator.expect_column_values_to_not_be_null("notes")

# Validate
results = validator.validate()


# Print results
print(results)

Calculating Metrics: 100%|██████████| 59/59 [00:03<00:00, 16.76it/s] 

{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "pandas-transactions_data",
          "column": "amount",
          "min_value": 0.01
        },
        "meta": {}
      },
      "result": {
        "element_count": 100000,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "pandas-transactions_data",
          "column": "tr